# Microsoft "is it malware" kaggle competition data

Link to kaggle comp [here](https://www.kaggle.com/c/microsoft-malware-prediction/data)

## The Data

This is a dataset collected by Microsoft from microsoft devices. The target is whether or not malware was detected on the machine by any means.

## The Business Problem

People host their lives on their devices, and those people will hold a company accountable for everything they can even if it was objectively the customers fault. Enter virus protection! Windows has their own proprietary AV software _Windows Defender_ though they usually recommend additional AV software.

In [1]:
#import the goods

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import eli5
import pdpbox
import numpy
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from eli5.sklearn import PermutationImportance
import sys
import copy
from pdpbox import pdp, get_dataset, info_plots
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn.metrics import plot_roc_curve
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
#if any packages fail to import (eli5 for example) try running !pip install "package-name" in its own cell block
# e.x !pip install eli5

#I reset this here because later we expand it to limitless and don't want to crash the system on repeat runs
pd.set_option('display.max_rows', 10)

In [2]:
#df_test = pd.read_csv('C:/Users/User/Desktop/msml-data/test.csv',nrows=100000)
#df_test

In [3]:
#importing first 100000 rows
df = pd.read_csv('C:/Users/User/Desktop/msml-data/train.csv',nrows=100000)

In [4]:
df = df.drop(columns='MachineIdentifier')

In [5]:
#df = df.dropna()

In [6]:
df.dtypes

ProductName                                 object
EngineVersion                               object
AppVersion                                  object
AvSigVersion                                object
IsBeta                                       int64
                                            ...   
Census_IsPenCapable                          int64
Census_IsAlwaysOnAlwaysConnectedCapable    float64
Wdft_IsGamer                               float64
Wdft_RegionIdentifier                      float64
HasDetections                                int64
Length: 82, dtype: object

In [7]:
df.isnull().sum()

ProductName                                   0
EngineVersion                                 0
AppVersion                                    0
AvSigVersion                                  0
IsBeta                                        0
                                           ... 
Census_IsPenCapable                           0
Census_IsAlwaysOnAlwaysConnectedCapable     843
Wdft_IsGamer                               3415
Wdft_RegionIdentifier                      3415
HasDetections                                 0
Length: 82, dtype: int64

In [8]:
df.shape

(100000, 82)

In [9]:
def thresh_col_remover(dataframe):
    '''Here we are removing largely null columns while also removing columns
    that contain a majority of one class cuz science'''
    #Boutta go ham on preprocessing, establishing variables to use for the goods
    #here we are saving the df size for use
    df_size = len(dataframe)
    #here I arbitrarily decide that if 80% of a column is NA it gets dropped for now
    #as pirated from kaggle
    na_cutoff = .8
    #feature cutoff as well, if there is a dominant feature it gets yoinked
    unbalanced_feature_cutoff = .95
    
    usable_features = list(dataframe.columns)
   
    for col in dataframe:
        
        current_col_distro = dataframe[col].value_counts(normalize=True, dropna=False).values[0]
        current_col_nulls = dataframe[col].isnull().sum() / df_size
        
        if current_col_nulls > na_cutoff:
        
            usable_features.remove(col)
        
        elif current_col_distro > unbalanced_feature_cutoff:
            
            usable_features.remove(col)
    
    return dataframe[usable_features], usable_features
ndf, usable_features = thresh_col_remover(df)

In [10]:
current_columns = df.columns.to_list()
#so we have usable columns and current columns. What is current - usable? 
# we lose the ones we want. but then we have a set of what?
#the ones we DONT want?
drop_these = list(set(current_columns)-set(usable_features))
#print(usable_features, '\n\n', drop_these)
#this looks about right for now

In [11]:
#I need to do this to have my new datframe (ndf) to have something to map to
df = df.drop(columns=drop_these)

In [12]:
#cols_removed = list(set(df.columns)-set(ndf.columns))
#cols_removed

In [13]:
#copying some jazz I seen on the internets
#filling NA values with the statistical mode
def mode_imputer(dataframe):
    mode_impute = SimpleImputer(strategy="most_frequent")
    dataframe = pd.DataFrame(mode_impute.fit_transform(dataframe))
    return dataframe
ndf = mode_imputer(ndf)
ndf.columns = usable_features
ndf

,EngineVersion,AppVersion,AvSigVersion,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,...,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsTouchEnabled,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,1.1.15100.1,4.18.1807.18075,1.273.1735.0,53447,1,29,128035,18,35,171,...,0,628,36144,0,0,0,0,0,10,0
1,1.1.14600.4,4.13.17134.1,1.263.48.0,53447,1,93,1482,18,119,64,...,0,628,57858,0,0,0,0,0,8,0
2,1.1.15100.1,4.18.1807.18075,1.273.1341.0,53447,1,86,153579,18,64,49,...,0,142,52682,0,0,0,0,0,3,0
3,1.1.15100.1,4.18.1807.18075,1.273.1527.0,53447,1,88,20710,27,117,115,...,0,355,20050,0,0,0,0,0,3,1
4,1.1.15100.1,4.18.1807.18075,1.273.1379.0,53447,1,18,37376,27,277,75,...,0,355,19844,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.1.15200.1,4.8.10240.17443,1.275.193.0,42086,2,177,75608,27,251,239,...,0,556,12456,1,0,0,0,0,15,0
99996,1.1.15100.1,4.18.1806.18062,1.273.371.0,53447,1,206,69788,27,276,74,...,0,500,43238,0,0,0,0,0,11,1
99997,1.1.15200.1,4.18.1807.18075,1.275.1244.0,7945,2,93,36825,27,277,75,...,0,355,19844,0,0,0,0,0,8,0
99998,1.1.15200.1,4.18.1807.18075,1.275.1362.0,53447,1,29,157243,18,35,171,...,0,807,52295,1,0,0,0,0,10,0


In [14]:
#here our datatypes are still cursed because of something numpy or arrays do under the hood
ndf.dtypes

EngineVersion                              object
AppVersion                                 object
AvSigVersion                               object
AVProductStatesIdentifier                  object
AVProductsInstalled                        object
                                            ...  
Census_IsTouchEnabled                      object
Census_IsAlwaysOnAlwaysConnectedCapable    object
Wdft_IsGamer                               object
Wdft_RegionIdentifier                      object
HasDetections                              object
Length: 62, dtype: object

# Feature engineering

## Cardinality

[Why I googled dataset cardinality](https://www.kaggle.com/fabiendaniel/detecting-malwares-with-lgbm) Kernel of fabiendaniel who did lgbm forward feature engineering

[An easy to digest explanation of cardinality using M&M's](https://blog.timescale.com/blog/what-is-high-cardinality-how-do-time-series-databases-influxdb-timescaledb-compare/)

If you are familiar with relational databases the concept of cardinality has a lot of overlap, things like one to many and one to one. In this example, about data-_sets_ with high cardinality it is simply a column with many unique values. And that can be dealt with through frequency encoding, _encode each unique value with its overall summed frequency in the dataset_.

In order to begin determining if we need to do this, we should check unique values of our categorical columns to see if we have high cardinality. Hopefully the sample of rows I chose can be considered representative of the set.

In [15]:
#here is one solution to the datatypes failing to copy another is to 
#copy the values as array and set the columns =
# ndf = pd.Dataframe(df.values.tolist(),columns = df.columns)
ndf = df.astype(df.dtypes.to_dict())

In [16]:
df.dtypes.to_dict()

{'EngineVersion': dtype('O'),
 'AppVersion': dtype('O'),
 'AvSigVersion': dtype('O'),
 'AVProductStatesIdentifier': dtype('float64'),
 'AVProductsInstalled': dtype('float64'),
 'CountryIdentifier': dtype('int64'),
 'CityIdentifier': dtype('float64'),
 'OrganizationIdentifier': dtype('float64'),
 'GeoNameIdentifier': dtype('float64'),
 'LocaleEnglishNameIdentifier': dtype('int64'),
 'Processor': dtype('O'),
 'OsBuild': dtype('int64'),
 'OsSuite': dtype('int64'),
 'OsPlatformSubRelease': dtype('O'),
 'OsBuildLab': dtype('O'),
 'SkuEdition': dtype('O'),
 'IsProtected': dtype('float64'),
 'SMode': dtype('float64'),
 'IeVerIdentifier': dtype('float64'),
 'SmartScreen': dtype('O'),
 'Census_MDC2FormFactor': dtype('O'),
 'Census_OEMNameIdentifier': dtype('float64'),
 'Census_OEMModelIdentifier': dtype('float64'),
 'Census_ProcessorCoreCount': dtype('float64'),
 'Census_ProcessorManufacturerIdentifier': dtype('float64'),
 'Census_ProcessorModelIdentifier': dtype('float64'),
 'Census_PrimaryDis

In [17]:
#experiencing a problem, I have lost my datatypes, probably happened in 
#function thresh_col_remover when I sent new columns to a dataframe?
typesDict = ndf.dtypes.to_dict()

In [18]:
#mildly copied from fabiendaniel
#grabbing numerical columns hopefully
type_numeric = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']

#can create iterable (list) here using this loop notation
numerical_columns = [c for c,v in typesDict.items() if v in type_numeric]
categorical_columns = [c for c,v in typesDict.items() if v not in type_numeric]
print(numerical_columns, '\n\n', categorical_columns)

['AVProductStatesIdentifier', 'AVProductsInstalled', 'CountryIdentifier', 'CityIdentifier', 'OrganizationIdentifier', 'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'OsBuild', 'OsSuite', 'IsProtected', 'SMode', 'IeVerIdentifier', 'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier', 'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier', 'Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTotalCapacity', 'Census_SystemVolumeTotalCapacity', 'Census_HasOpticalDiskDrive', 'Census_TotalPhysicalRAM', 'Census_InternalPrimaryDiagonalDisplaySizeInInches', 'Census_InternalPrimaryDisplayResolutionHorizontal', 'Census_InternalPrimaryDisplayResolutionVertical', 'Census_InternalBatteryNumberOfCharges', 'Census_OSBuildNumber', 'Census_OSBuildRevision', 'Census_OSInstallLanguageIdentifier', 'Census_OSUILocaleIdentifier', 'Census_ThresholdOptIn', 'Census_FirmwareManufacturerIdentifier', 'Census_FirmwareVersionIdentifier', 'Census_IsSecureBootEnabled', 'Census_IsWIMBootEnabled'

In [19]:
#observing amount of unique values per categorical column, that cardinality tho
#comparing these results of my first nrows to the other kernels this is mildly unrepresentative but it's just an example
#I will be frequency encoding the columns with high cardinality (unique values) per linked kernel by fabienDaniel
#https://www.kaggle.com/fabiendaniel/detecting-malwares-with-lgbm
for col in categorical_columns:
    print(col, len(ndf[col].value_counts().unique()))

EngineVersion 37
AppVersion 59
AvSigVersion 247
Processor 3
OsPlatformSubRelease 9
OsBuildLab 107
SkuEdition 8
SmartScreen 11
Census_MDC2FormFactor 12
Census_PrimaryDiskTypeName 4
Census_ChassisTypeName 23
Census_PowerPlatformRoleName 9
Census_InternalBatteryType 16
Census_OSVersion 139
Census_OSArchitecture 3
Census_OSBranch 14
Census_OSEdition 18
Census_OSSkuName 16
Census_OSInstallTypeName 9
Census_OSWUAutoUpdateOptionsName 6
Census_GenuineStateName 4
Census_ActivationChannel 6
Census_FlightRing 6


In [20]:
frequency_encoded_variables = [
    'Census_OEMModelIdentifier',
    'CityIdentifier',
    'Census_FirmwareVersionIdentifier',
    'AvSigVersion',
    'Census_ProcessorModelIdentifier',
    'Census_OEMNameIdentifier',
    'DefaultBrowsersIdentifier'
]

In [21]:
#here I am just replacing ndf new data frame with train for readability
train = ndf
train.dtypes

EngineVersion                               object
AppVersion                                  object
AvSigVersion                                object
AVProductStatesIdentifier                  float64
AVProductsInstalled                        float64
                                            ...   
Census_IsTouchEnabled                        int64
Census_IsAlwaysOnAlwaysConnectedCapable    float64
Wdft_IsGamer                               float64
Wdft_RegionIdentifier                      float64
HasDetections                                int64
Length: 62, dtype: object

In [22]:
y = ndf['HasDetections']
X = ndf.drop(['HasDetections'], axis=1)

In [23]:
#skf.get_n_splits(X,y)

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.30, random_state=1)